In [22]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Feb 10, 2020
@author: neeks, cmu
"""
import glob # for_listing_filenames_from_directory
import os # for_extracting_filename_from_path
import librosa # audio_processing_library
from yaafelib import *
import numpy as np
import pickle
import matplotlib.pyplot as plt
# Build a DataFlow object using FeaturePlan

In [27]:
path_wav_files = '../lisTest/tcdDecision/playback/STIMULI/'
wav_folder_types = ['chinese','english']
file_types = ['nspkrs_1','nspkrs_2']

path_store_feats = './data/feats/'
feats_type = ['mfcc', 'mfcc_d1','mfcc_d2']

In [28]:
# init feature params
fp = FeaturePlan(sample_rate=16000)
fp.addFeature('mfcc: MFCC blockSize=400 stepSize=160 CepsIgnoreFirstCoeff=1 CepsNbCoeffs=12')
fp.addFeature('mfcc_d1: MFCC blockSize=400 stepSize=160 CepsIgnoreFirstCoeff=1 CepsNbCoeffs=12 > Derivate DOrder=1')
fp.addFeature('mfcc_d2: MFCC blockSize=400 stepSize=160 CepsIgnoreFirstCoeff=1 CepsNbCoeffs=12 > Derivate DOrder=2')


# init yaafe engine
engine = Engine()
engine.load(fp.getDataFlow())
engine.getInputs()
{'audio': {'sampleRate': 16000.0,
           'frameLength': 1,
           'sampleStep': 1,
           'parameters': {'SampleRate': '16000'},
           'size': 1}}

{'audio': {'sampleRate': 16000.0,
  'frameLength': 1,
  'sampleStep': 1,
  'parameters': {'SampleRate': '16000'},
  'size': 1}}

In [29]:
# read wav filenames with path
wav_files = []
for i in range(len(wav_folder_types)):
    wav_files.append([])
    for j in range(len(file_types)):
        wav_files[i].append([])
        for file in glob.glob(path_wav_files+wav_folder_types[i]+'/wavFilesTest/'+'*'+file_types[j]+'.wav'):
            wav_files[i][j].append(file)

In [30]:
# extract and store feats as pickle file
for i in range(len(wav_files)):
    for j in range(len(wav_files[i])):
        for k in range(len(wav_files[i][j])):
            # read wav file
            y, sr = librosa.load(wav_files[i][j][k],sr=None)
            head, tail = os.path.split(wav_files[i][j][k])
            print(str(i+1)+': Extracting features from: ' + tail)
            if sr != 16000:
                y = librosa.resample(y, sr, 16000)
                sr = 16000
            y = y/np.max(abs(y))

            # extract features from a numpy array
            audio = np.reshape(y,(1,len(y)))
            audio = np.float64(audio)
            feats = engine.processAudio(audio)

            for l in range(len(feats_type)):
                if not os.path.exists(path_store_feats+feats_type[l]):
                    os.mkdir(path_store_feats+feats_type[l])
                    with open(path_store_feats+feats_type[l]+'/'+tail[:-4]+'_'+feats_type[l]+
                              '.pickle', 'wb') as f:
                        pickle.dump(feats[feats_type[l]], f)
                    print(1/)

1: Extracting features from: BAC009S0128W0416_BAC009S0128W0353_02_59_nspkrs_1.wav


ZeroDivisionError: division by zero